In [1]:
#linear algebra
import numpy as np
#data structures
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('big_data.csv')


# city_to_area = {}
# missing_cities = []


# for i in hh_dt.area:
#     city_rows = data[data.unified_address_city == i].unified_address_state.to_list()
#     if len(city_rows) == 0:
#         missing_cities.append(i)
#         continue
#     city_to_area[i] = city_rows[0]


# hh_dt.drop(index=hh_dt[hh_dt.area.isin(missing_cities)].index, inplace=True)
# hh_dt.area = hh_dt.area.map(city_to_area)

dt = data[['experience_name', 'schedule_name', 'unified_address_state', 'professional_roles_name', 'salary_mean_net',]]


In [4]:
dt['schedule_name'].replace('Сменный график', 0, inplace=True)
dt['schedule_name'].replace('Полный день', 1, inplace=True)
dt['schedule_name'].replace('Гибкий график', 2, inplace=True)
dt['schedule_name'].replace('Вахтовый метод', 3, inplace=True)
dt['schedule_name'].replace('Удаленная работа', 4, inplace=True)

dt['experience_name'].replace('Нет опыта', 0, inplace=True)
dt['experience_name'].replace('От 1 года до 3 лет', 1, inplace=True)
dt['experience_name'].replace('От 3 до 6 лет', 2, inplace=True)
dt['experience_name'].replace('Более 6 лет', 3, inplace=True)

In [5]:
states = []

for i in dt.unified_address_state:
  if i not in states and isinstance(i, int) is False:
    states.append(i)
    dt['unified_address_state'].replace(i, len(states) - 1, inplace=True)



In [6]:
prof = []

for i in dt.professional_roles_name:
  if i not in prof and isinstance(i, int) is False:
    prof.append(i)
    dt['professional_roles_name'].replace(i, len(prof) - 1, inplace=True)


In [7]:
roles_to_drop = [5]

dt.drop(dt[dt.professional_roles_name.isin(roles_to_drop)].index, inplace=True)
#удаляем строчки, проффесии которых встречаются до 50 раз

ser = dt.unified_address_state.value_counts()
states_to_drop = list(ser[ser < 50].index)

dt.drop(dt[dt.unified_address_state.isin(states_to_drop)].index, inplace=True)
#удаляем строчки, округов которые встречаются до 50 разb

## train

In [8]:
dt.drop(dt[dt.salary_mean_net.isna()].index, inplace=True)

In [9]:
X = dt.drop(['salary_mean_net',], axis=1)
Y = dt.salary_mean_net

# X = np.concatenate( (X, vectors_transformed) , axis=1)

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)


In [22]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

def train_model(model):
  model.fit(x_train, y_train)
  preds = model.predict(x_test)
  print('model:', type(model))
  print('mse:', mean_absolute_error(y_test, preds))
  print('r2:', r2_score(y_test, preds))
  print('mape:', mean_absolute_percentage_error(y_test, preds))
  return preds




In [11]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(max_depth=4, random_state=17, criterion="absolute_error")
train_model(model)

model: <class 'sklearn.tree._classes.DecisionTreeRegressor'>
mse: 17995.359726161732
r2: 0.22049057000265848
mape: 0.3574432894295944


In [23]:
from sklearn.linear_model import LinearRegression

modal = LinearRegression()
preds = train_model(modal)


model: <class 'sklearn.linear_model._base.LinearRegression'>
mse: 19990.872638761124
r2: 0.18343448387966066
mape: 0.4702282399573613


In [25]:
preds

array([68833.61130806, 55038.57127081, 53655.94908594, ...,
       62565.0973398 , 36126.80770216, 24434.66170526])

In [24]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

ValueError: continuous is not supported

In [13]:
from sklearn.ensemble import RandomForestRegressor

modal = RandomForestRegressor(max_depth=4, random_state=0)
train_model(modal)

model: <class 'sklearn.ensemble._forest.RandomForestRegressor'>
mse: 18703.649209911193
r2: 0.2908301788461626
mape: 0.4320141616754631


In [14]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor()

train_model(model)

model: <class 'sklearn.ensemble._gb.GradientBoostingRegressor'>
mse: 16595.31016831857
r2: 0.39537247987651836
mape: 0.3754572440912871
